In [1]:
import numpy as np
import pandas as pd
import pwlf
import io
import matplotlib.pyplot as plt
from ipywidgets import interactive, GridBox, Layout, IntSlider, interactive_output, Select, FileUpload, Dropdown
from IPython.display import display
import warnings
import voila

In [4]:
def plot_item(history, forecast, breakpoints, group, gvalue, x, y, y_max):
    global xlabel, ylabel
    
    if x is None or y is None or group is None:
        return
    
    if group == '_None_' or x == '_None_' or y == '_None_':
        return

    gv = gvalue
    
    mask = (df[group] == gv)
    x = df[mask][x].tail(history)
    x = np.arange(0, len(x))
    y = df[mask][y].tail(history)
    y = y.fillna(method='ffill')
    y = y.fillna(method='bfill')
    
    y_size = None
    
    if y_max is not None and y_max != '_None_':
        y_size = df[mask][y_max].tail(history)
        y_size = y_size.fillna(method='ffill')
    
    model = pwlf.PiecewiseLinFit(x, y)
    

    model.fitfast(breakpoints)

    r_squared = model.r_squared()
    x_hat = x
    y_hat = model.predict(x_hat)
    # calculate the prediction variance at the xHat locations
    pre_var_x_hat = model.prediction_variance(x_hat)
    # turn variance into standard deviation
    pre_sigma_x_hat = np.sqrt(pre_var_x_hat)

    x_hat_f = np.arange(x_hat.max(), len(x_hat) + forecast + 1, 1)
    y_hat_f = model.predict(x_hat_f)
    
    # calculate the prediction variance at the xHat locations
    pre_var_x_hat_f = model.prediction_variance(x_hat_f)
    # turn variance into standard deviation
    pre_sigma_x_hat_f = np.sqrt(pre_var_x_hat_f)

    fig, ax = plt.subplots(figsize=(20, 10))
    plt.plot(x, y, 'o', )
    
    if y_size is not None:
        plt.plot(x, y_size, '-', c='red', lw=3)
        y_size_avail = np.repeat(y_size.tail(1).values, len(x_hat_f))
        plt.plot(x_hat_f, y_size_avail, '-', c='red', lw=3)
    
    plt.plot(x_hat, y_hat, '-', lw=3)
    y_hat_hist_cf_min = y_hat - (pre_sigma_x_hat*1.96)
    y_hat_hist_cf_max = y_hat + (pre_sigma_x_hat*1.96)
    plt.fill_between(x_hat, y_hat_hist_cf_min, y_hat_hist_cf_max,
                     alpha=0.1, color="r")
    plt.plot(x_hat_f, y_hat_f, '--', lw=3)
    
    y_hat_proj_cf_min = y_hat_f - (pre_sigma_x_hat_f*1.96)
    y_hat_proj_cf_max = y_hat_f + (pre_sigma_x_hat_f*1.96)
    
    plt.fill_between(x_hat_f, y_hat_proj_cf_min, y_hat_proj_cf_max,
                     alpha=0.1, color="g")
    
    plt.text(0.5, 1.05, f"Item: {gv}\nHistory: {history} Forecast: {forecast} " \
                        f"Breakpoints: {breakpoints} " + r"$R^2$" + f": {r_squared:0.2f}",
        fontsize=20, family='monospace',
        transform=fig.transFigure, horizontalalignment='center', verticalalignment='top'
    )

    if y_size is not None:
        y_hat_proj_cf_min = int(y_hat_proj_cf_min[-1])
        y_hat_proj_cf_max = int(y_hat_proj_cf_max[-1])

        proj_range = y_hat_proj_cf_max - y_hat_proj_cf_min

        days_full_min = None
        days_full_max = None

        m = np.vstack((y_hat_f + (pre_sigma_x_hat_f*1.96), y_size_avail))
        m_res = m[0] >= m[1]
        if m_res.any():
            days_full_min = m_res.argmax() - 1
            if days_full_min < 0: days_full_min = 0

        m = np.vstack((y_hat_f - (pre_sigma_x_hat_f*1.96), y_size_avail))
        m_res = m[0] >= m[1]
        if m_res.any():
            days_full_max = m_res.argmax() - 1
            if days_full_max < 0: days_full_max = 0

        if days_full_min != None:
            if days_full_max == None:
                days_full_max = f"{forecast}+"
            plt.text(0.5, 0.0, f"Predict system will fill up within {days_full_min} and {days_full_max} points\n" \
                               f"To FULL Min:Max:Range: {y_hat_proj_cf_min}:{y_hat_proj_cf_max}:{proj_range}",
                     bbox=dict(facecolor='red', alpha=0.5), color="white", fontsize=20, family='monospace',
                     transform=fig.transFigure, horizontalalignment='center', verticalalignment='top')

    plt.grid(which='both', axis='y')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xticks([], [])
    plt.xticks(range(x_hat.min(), x_hat_f.max()+1), range(x_hat.min(), x_hat_f.max()+1), rotation=90)


# Interactive Forecast Tool

In [6]:
%matplotlib inline

def fu_on_change(n):
    global df, group, x, y, y_max
    
    if n['new'] == n['old']:
        return

    filename = tuple(n['new'].keys())[0]
    ext = filename.split('.')[-1].lower()

    pd_read = pd.read_table
    if ext == 'xlsx':
        pd_read = pd.read_excel

    df = pd_read(io.BytesIO(n['new'][filename]['content']))
    scols = ['_None_', *df.columns.tolist()]
    
    group.options = scols
    x.options = scols
    y.options = scols
    y_max.options = scols

def group_on_change(n):
    global df, gvalue
    
    if n['new'] == n['old']:
        return
    
    gv = n['new']
    
    if gv in df.columns:
        gvalue.options = df[gv].unique()
    else:
        gvalue.options = []

def x_on_change(n):
    global xlabel
    
    if n['new'] == n['old']:
        return
    
    if n['new']:
        xlabel = n['new']

def y_on_change(n):
    global ylabel
    
    if n['new'] == n['old']:
        return
    
    if n['new']:
        ylabel = n['new']

warnings.filterwarnings("ignore")

xlabel = ''
ylabel = ''

fu = FileUpload(description="Upload Spreadsheet or CSV", layout=Layout(width='auto', grid_area='fu'))
fu.observe(fu_on_change, names="value")
    
history = IntSlider(30, 2, 30*12, 7, description="History", layout=Layout(width='auto', grid_area='history'), continuous_update=False)
forecast = IntSlider(30, 2, 30*12,7, description="Forecast", layout=Layout(width='auto', grid_area='forecast'), continuous_update=False)
breakpoints = IntSlider(2, 2, 10, 1, description="Breakpoints", layout=Layout(width='auto', grid_area='breakpoints'), continuous_update=False)
gvalue = Select(description="Item", options=[], layout=Layout(width='auto', grid_area='gvalue'), continuous_update=False)

group = Dropdown(description="Group", options=[], layout=Layout(width='auto', grid_area='group'))

group.observe(group_on_change, names="value")

x = Dropdown(description="X", options=[], layout=Layout(width='auto', grid_area='x'))
xlabel = x.observe(x_on_change, 'value')

y = Dropdown(description="Y", options=[], layout=Layout(width='auto', grid_area='y'))
ylabel = y.observe(y_on_change, 'value')

y_max = Dropdown(description="Y_MAX", options=[], layout=Layout(width='auto', grid_area='y_max'))

graph = interactive_output(plot_item, {"history":history, "forecast":forecast,
                                     "breakpoints":breakpoints,
                                     "group": group, "gvalue": gvalue, "x": x, "y": y, "y_max": y_max})
graph.layout.grid_area = "graph"
graph.layout.width = "auto"
graph.layout.justify_content = "center"
graph.layout.justify_items = "center"

display(
    GridBox(
        children=[fu, group, x, y, y_max, gvalue, history, forecast, breakpoints, graph],
        layout=Layout(
            width='100%',
            grid_template_rows='auto auto auto auto auto auto auto auto auto',
            grid_template_columns='40% 50%',
            grid_template_areas='''
            "fu graph"
            "group graph"
            "x graph"
            "y graph"
            "y_max graph"
            "gvalue graph"
            "history graph"
            "forecast graph"
            "breakpoints graph"
            ''')
       )
);

GridBox(children=(FileUpload(value={}, description='Upload Spreadsheet or CSV', layout=Layout(grid_area='fu', …